# RuntimeError: CUDA out of memory. 

关于CUDA GPU显存管理的总结：
- GPU显存占用率和存入的数据尺寸成正相关，越大的数据占用显存越多
- 只要使用了GPU，就至少会占x xx M的显存，且这部分显存无法被释放
- 当一块内存不再被变量所引用时，这块内存就由激活内存转为失活内存，但它仍然存在于这个数据队列中
- 当数据队列达到某个阈值时，CUDA会触发垃圾回收机制，清理失活内存
- 运行torch.cuda.empty_cache()可以手动清理失活内存

那么根据上述理论，就可以得到对应的问题解决方案：
- 调小batch_size  
  本质上是防止GPU数据队列向显存申请的空间大于显存本身

- 检查是否有数据持续存入GPU而未释放

- 训练过程中的测试阶段和验证阶段前插入代码with torch.no_grad()  
  原理是不计算梯度，从而不用GPU加速运算，不会把数据再加到数据队列中